In [1]:
import math
from scipy.integrate import quad, fixed_quad
import matplotlib.pyplot as plt
import pickle 
import numpy as np
import functools
import sys 
from tqdm import tqdm, trange
from utils import *
from classy import Class

import math
import torch
import gpytorch
from matplotlib import pyplot as plt

In [14]:
cosmo_params['Box0_1400'].keys()

dict_keys(['As', 'ns', 'H0', 'w0', 'ombh2', 'omch2', 'nu_mass_ev', 'sigma'])

In [18]:
cosmo_params['Box_n50_0_1400']

{'As': 2.10100315,
 'ns': 0.97000003,
 'H0': 67.0,
 'w0': -1.0,
 'ombh2': 0.0223,
 'omch2': 0.12,
 'nu_mass_ev': 0.07071068,
 'sigma': 0.81001457}

In [2]:
cosmos_f = open('data/cosmo_params.pkl', 'rb')
cosmo_params = pickle.load(cosmos_f) #cosmo_params is a dict
cosmos_f.close()

a_list_fname = '/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/alist.pkl'
a_list_f = open(a_list_fname, 'rb')
a_list = pickle.load(a_list_f) 
a_list_f.close()

In [3]:
weird_boxes = ['Box63_1400', 'Box35_1400', 'Box_n50_38_1400', 'Box5_1400']

In [4]:
errors = {a:{} for a in a_list}
X = []
Y = []
z_to_a = {}
a_to_z = {}
kt = np.logspace(-3, 1, 100) # h/Mpc
for box in tqdm(cosmo_params):
    if(box in weird_boxes):
        continue
    curr_cosmo = cosmo_params[box]
    curr_cosmo_values = list(curr_cosmo.values())
    
    h = curr_cosmo['H0']/100

    cosmo_dict = {
        'h': h,
        'Omega_b': curr_cosmo['ombh2'] / h**2,
        'Omega_cdm': curr_cosmo['omch2'] / h**2,
        'N_ur': 0.00641,
        'N_ncdm': 1,
        'output': 'mPk mTk',
        'z_pk': '0.0,99',
        'P_k_max_h/Mpc': 20.,
        'm_ncdm': curr_cosmo['nu_mass_ev']/3,
        'deg_ncdm': 3,
        'T_cmb': 2.7255,
        'A_s': curr_cosmo['As'] * 10**-9,
        'n_s': curr_cosmo['ns'],
        'Omega_Lambda': 0.0,
        'w0_fld': curr_cosmo['w0'],
        'wa_fld': 0.0,
        'cs2_fld': 1.0,
        'fluid_equation_of_state': "CLP"
    }

    pkclass = Class()
    pkclass.set(cosmo_dict)
    pkclass.compute()

    for a in a_list:
        z = scaleToRedshift(a)
        z_to_a[z] = a
        a_to_z[a] = z
        pk_m_lin = np.array(
            [
                pkclass.pk_lin(ki, np.array([z]))*h**3 #units of Mpc^3/h^3
                for ki in kt * h # 1 / Mpc
            ]
        )
        sigma8 = pkclass.sigma(8, z, h_units=True)
        X+= [curr_cosmo_values + [a, sigma8]]
        with open("/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/%s_%.2f_params.pkl"%(box, a), "rb") as f:
            MLE_params = pickle.load(f)
            param_values = list(MLE_params.values())
            Y+= [param_values]
X = np.array(X)
Y = np.array(Y)

100%|██████████| 150/150 [03:15<00:00,  1.30s/it]


In [6]:
print('scaling input')
in_scaler = Normalizer()
in_scaler.fit(X)
X = in_scaler.transform(X)
print(X.shape)


print('scaling output')
out_scaler = Standardizer()
out_scaler.fit(Y)
Y = out_scaler.transform(Y)
print(Y.shape)

scaling input
(2044, 10)
scaling output
(2044, 4)


In [7]:
X_train = torch.from_numpy(X).float()
Y_train = torch.from_numpy(Y).float()
n_tasks = len(Y_train[0])

class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.LinearMean(input_size=X_train.shape[1]), num_tasks=n_tasks
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
             gpytorch.kernels.SpectralMixtureKernel(num_mixtures=3, 
                                                    ard_num_dims=X_train.shape[1]),
            num_tasks=n_tasks, rank=1
        )
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)

In [8]:
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=n_tasks)
model = MultitaskGPModel(X_train, Y_train, likelihood)


# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.1, amsgrad=True)  # Includes GaussianLikelihood parameters

In [9]:
from copy import deepcopy
model.train()
likelihood.train()
# Set initial learning rate
lr = 0.1

# Create the optimizer with the initial learning rate
optimizer = torch.optim.AdamW(model.parameters(), lr=0.1, amsgrad=True)  # Includes GaussianLikelihood parameters

training_iterations = 300
epochs_iter = tqdm(range(training_iterations), desc="Iteration")

Iteration:   0%|          | 0/300 [00:00<?, ?it/s]

In [10]:
for i in epochs_iter:
    model.train()
    likelihood.train()

    optimizer.zero_grad()
    output = model(X_train)
    loss = -mll(output, Y_train)
    epochs_iter.set_postfix(loss=loss.item())
    loss.backward()
    optimizer.step()

    # Change learning rate after half of iterations
    if i == training_iterations//2:
        lr = 0.01
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr


Iteration: 100%|██████████| 300/300 [14:19<00:00,  2.86s/it, loss=-.806] 


In [17]:
model.eval()
likelihood.eval()

with open("GP_emulator.pkl", "wb") as f:
    pickle.dump([model,
                in_scaler,
                out_scaler,
                likelihood], f)